Creo un dictionary in cui riporto tutti i modelli che andrò ad analizzare

In [7]:
import xarray as xr
import numpy as np
import os
#from cdo import *
from funzioni import compute_dataset_weighted
from funzioni import compute_annual_mean_weighted

In [52]:
#cdo = Cdo()

In [2]:
# creo un dizionario in cui andrò a mettere tutti i modelli dentro
models = {}

In [9]:
#Costruisco il dizionario models con il nome del modello, il nomde del percorso di tutti i file *.nc di ogni modello e il nome di .nc, cioè il modello remapbil

# percorso dei file model.txt e tos_nc_model.txt
name_model_path = '/work/users/guest/sbeltrami/name_model.txt'
tos_nc_path = '/work/users/guest/sbeltrami/tos_nc_model.txt'
remap_tos_nc_path = '/work/users/guest/sbeltrami/remap_tos_nc_model.txt'
# Apro i file in modalità di lettura
with open(name_model_path, 'r') as file:
    # Leggo tutte le righe del file e salvo in una lista
    lines_name_model = file.readlines()
with open(tos_nc_path, 'r') as file:
    # Leggo tutte le righe del file e salvo in una lista
    lines_tos_nc = file.readlines()
with open(remap_tos_nc_path, 'r') as file:
    lines_remap_tos_nc = file.readlines()


# Itero su tutte le righe e organizzo i dati nel dizionario
for line_name, line_tos, line_remap in zip(lines_name_model, lines_tos_nc, lines_remap_tos_nc):
    model_name = line_name.strip()

    # La chiave principale model_name deve essere presente nel dizionario
    if model_name not in models:
        models[model_name] = {}

    # Aggiungo i dati come sottochiavi
    models[model_name]['name'] = model_name
    models[model_name]['path'] = line_tos.strip()
    models[model_name]['path remap'] = line_remap.strip()

In [5]:
models['TaiESM1'].items()

dict_items([('model name', 'TaiESM1'), ('model path', '/work/users/guest/sbeltrami/TaiESM1.nc'), ('model path remap', '/work/users/guest/sbeltrami/TaiESM1_remap.nc')])

In [10]:
count = 0 #conteggio dei file che non si possono aprire
count_ok = 0 #conteggio dei file che apro

for name in models.keys():
    # Verifica se il file NetCDF esiste
    #path_remap = models[name]['path remap']
    if os.path.exists(models[name]['path remap']):
        # Se il file esiste, apri il dataset con xarray
        dataset = xr.open_dataset(models[name]['path remap'])
        print(dataset)
        # Continua con le operazioni sul dataset
        print(f"Dataset aperto con successo per il modello {name}")
        #peso il dataset
        models[name]['dataset weighted'] = compute_dataset_weighted(dataset["tos"])
        print("ok")
        #medie annuali con dataset pesato
        #models[name]['annual mean'] = compute_annual_mean_weighted(models[name]['dataset weighted'])
        count_ok = count_ok + 1
    else:
        print(f"Errore: Il file {models[name]['path remap']} non esiste per il modello {name}")
        count = count + 1


<xarray.Dataset>
Dimensions:    (time: 1980, bnds: 2, lon: 180, lat: 90)
Coordinates:
  * time       (time) object 1850-01-17 00:30:00.000003 ... 2014-12-16 12:00:00
  * lon        (lon) float64 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * lat        (lat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    tos        (time, lat, lon) float32 ...
Attributes: (12/48)
    CDI:                    Climate Data Interface version 2.0.4 (https://mpi...
    source:                 TaiESM 1.0 (2018): \naerosol: SNAP (same grid as ...
    institution:            Research Center for Environmental Changes, Academ...
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          Hybrid-restart from year 0671-01-01 of piControl
    ...                     ...
    variable_id:            tos
    variant_label:          r1i1p1f1
    license:              

KeyError: 'tos'

In [28]:
n = len(models) #numero di modelli

67

In [33]:
models.items()

dict_items([('TaiESM1', {'model name': 'TaiESM1'}), ('AWI-CM-1-1-MR', {'model name': 'AWI-CM-1-1-MR'}), ('AWI-ESM-1-1-LR', {'model name': 'AWI-ESM-1-1-LR'}), ('BCC-CSM2-MR', {'model name': 'BCC-CSM2-MR'}), ('BCC-ESM1', {'model name': 'BCC-ESM1'}), ('CAMS-CSM1-0', {'model name': 'CAMS-CSM1-0'}), ('CAS-ESM2-0', {'model name': 'CAS-ESM2-0'}), ('FGOALS-f3-L', {'model name': 'FGOALS-f3-L'}), ('FGOALS-g3', {'model name': 'FGOALS-g3'}), ('CanESM5', {'model name': 'CanESM5'}), ('CanESM5-1', {'model name': 'CanESM5-1'}), ('IITM-ESM', {'model name': 'IITM-ESM'}), ('CMCC-CM2-HR4', {'model name': 'CMCC-CM2-HR4'}), ('CMCC-CM2-SR5', {'model name': 'CMCC-CM2-SR5'}), ('CMCC-ESM2', {'model name': 'CMCC-ESM2'}), ('CNRM-CM6-1', {'model name': 'CNRM-CM6-1'}), ('CNRM-CM6-1-HR', {'model name': 'CNRM-CM6-1-HR'}), ('CNRM-ESM2-1', {'model name': 'CNRM-ESM2-1'}), ('ACCESS-ESM1-5', {'model name': 'ACCESS-ESM1-5'}), ('ACCESS-CM2', {'model name': 'ACCESS-CM2'}), ('MPI-ESM1-2-HR', {'model name': 'MPI-ESM1-2-HR'}), 

In [ ]:
tos = xr.open_mfdataset(paths="/archive/paolo/cmip6/CMIP6/model-output/EC-Earth-Consortium/EC-Earth3/historical/ocean/Omon/r1i1p1f1/tos/*.nc")

In [ ]:
new_tos= xr.open_dataset('/work/users/guest/sbeltrami/tostest.nc') #tostest.nc contiene tutti i dati concatenati in tos, usando il comando: cdo remapbil,r180x90 -cat *.nc /work/users/guest/sbeltrami/tostest.nc

prova con soli 3 modelli

In [5]:
models = { 'Ec-Earth3':
                    {'path': '/work/users/guest/sbeltrami/tostest.nc',
                     },
            'IPSL-CM6A-LR':
                    {'path': '/work/users/guest/sbeltrami/tostest_ipsl.nc',
                     },
            'MPI-ESM1-2-HR':
                    {'path': '/work/users/guest/sbeltrami/tostest_mpi.nc',
                     }
         }

In [6]:
for name in models.keys():
    #apertura modelli
    dataset = xr.open_dataset(models[name]['path'])
    #peso il dataset
    models[name]['dataset weighted'] = compute_dataset_weighted(dataset["tos"])
    #medie annuali con dataset pesato
    models[name]['annual mean'] = compute_annual_mean_weighted(models[name]['dataset weighted'])


In [9]:
models['Ec-Earth3'].keys()

dict_keys(['path', 'dataset weighted', 'annual mean'])

In [10]:
models['Ec-Earth3'].items()